In [ ]:
%pylab inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import datetime as dt

In [ ]:
import yatsm
from yatsm.io import read_pixel_timeseries
from yatsm.utils import csvfile_to_dataframe, get_image_IDs
from yatsm.config_parser import convert_config, parse_config_file
from yatsm.config_parser import convert_config, parse_config_file
import yatsm._cyprep as cyprep

Read in config file and dates:

In [ ]:
## SPECIFY CONFIG FILE
config_file = '/projectnb/landsat/projects/Massachusetts/p012r031/p012r031_config.yaml'

# Read in and parse config file
cfg = parse_config_file(config_file)

In [ ]:
# Get files list
df = csvfile_to_dataframe(cfg['dataset']['input_file'], \
                          date_format=cfg['dataset']['date_format'])

In [ ]:
# Get dates
df['image_ID'] = get_image_IDs(df['filename']) 
df['x'] = df['date'] 
dates = df['date'].values

Specify pixel to run:

In [ ]:
# SPECIFY PIXEL TO RUN
py = 4538 # row
px = 4622 # column

Configure model and read in time series:

In [ ]:
# Configure model
#model = cfg['YATSM']['algorithm_cls']
#algo_cfg = cfg[cfg['YATSM']['algorithm']]
#yatsm = model(estimator=cfg['YATSM']['estimator'], \
#                  **algo_cfg.get('init', {}))
#yatsm.px = px
#yatsm.py = py

In [ ]:
# Setup algorithm and create design matrix (if needed)
#X = yatsm.setup(df, **cfg)
#design_info = getattr(X, 'design_info', None)

In [ ]:
# Read in time series as numpy array
Y = read_pixel_timeseries(df['filename'], px, py)

Mask time series based on physical constraints and Fmask:

In [ ]:
# Mask based on physical constraints and Fmask 
valid = cyprep.get_valid_mask( \
            Y, \
            cfg['dataset']['min_values'], \
            cfg['dataset']['max_values']).astype(bool)

# Apply mask band
idx_mask = cfg['dataset']['mask_band'] - 1
valid *= np.in1d(Y.take(idx_mask, axis=0), \
                         cfg['dataset']['mask_values'], \
                         invert=True).astype(np.bool)

Y = np.delete(Y, idx_mask, axis=0)[:, valid]
#X = X[valid, :]
dates = dates[valid]

Plot example time series:

In [ ]:
# convert time from ordinal to dates
dt_dates = np.array([dt.datetime.fromordinal(d) for d in dates])

In [ ]:
# Plot time series - visual check
band = 9

plt.figure(figsize=(16,4),facecolor='w', edgecolor='k')
plt.plot(dt_dates, Y[band, :],'ko')
plt.title([py, px])
plt.show()

Correlation Analysis:

In [ ]:
# Read np array into pandas dataframe - exclude Fmask result
Y_df = pd.DataFrame(Y[0:10])

In [ ]:
# Transpose time series dataframe for correlation analysis
Y_t = Y_df.transpose()
Y_t.columns = ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', \
                        'Thermal', 'Brightness', 'Greenness', 'Wetness'] 

In [ ]:
# Calculate pearson correlation matrix
corr_df = Y_t.corr(method='pearson')
corr_df

In [ ]:
# Create a mask to display only the lower triangle of the matrix (since it's mirrored around its 
# top-left to bottom-right diagonal).
mask = np.zeros_like(corr_df)
mask[np.triu_indices_from(mask, 1)] = True

# Create the heatmap using seaborn library. 
# List if colormaps (parameter 'cmap') is available here: http://matplotlib.org/examples/color/colormaps_reference.html

plt.figure(figsize=(16,12),facecolor='w', edgecolor='k')
seaborn.heatmap(corr_df, cmap='seismic', vmax=1.0, vmin=-1.0, linewidths=2.5, mask = mask, square=True)
seaborn.heatmap(corr_df, cmap='gist_rainbow', cbar=False, annot=True, fmt='0.2f', annot_kws={"size": 20}, alpha=0.0, square=True)

# Show the plot we reorient the labels for each column and row to make them easier to read.
cax = plt.gcf().axes[-1]
cax.tick_params(labelsize=24)

plt.yticks(rotation=0, size=24) 
plt.xticks(rotation=90, size=24)
plt.title("Pearson's Correlation", size=30, weight='bold')
plt.show()